

### 1
**同时考虑分段里程的距离、转弯角度、方向，不把完全匹配值设为1.0，将匹配得分与负样例比较得到的匹配得分相比较，计算相应的概率。除了里程匹配，还有其他特征包括速度，加速度，停车前速度等等。GBDT.**<br/>
When comparing trip parts, my distance metric took into account both the distances between turns and the angles (I didn't ignore left/right, so I checked both the original angles and the angles*(-1) ). I didn't set all matches to 1.0; instead, I compared the match score to the scores obtained when comparing the trip to negative samples (i.e., trips from other drivers), and computed a corresponding probability (p-value).
### 2
**首先在里程匹配阶段计算每个里程匹配的路线；对所有匹配到两个或以上里程的路线，将它们放到顶端，以它们匹配到的其他里程排序，再按telematic的结果排序，得的约40万个里程；对于另外14万7千个里程，它们是否能够影响telematic基于它们是否存在里程匹配：如果有，调高telematic得分；如果是长里程，并且没有里程匹配，调低在telematic的结果。**<br/>
-  In the trip matching find out how many routes each trip matches

-  For routes matching 2 or more trips, set them at the top, ranked by the number of other trips that they matched, and then sorted by the telematics results.  For us, this was approx 400,000 trips

-for the other 147,000 trips, influence the telematics results based on if there was a trip match.

    - If there was a trip match, bump the telematics score up

    -  if it was a long run   (i.e. not the shortest 20% of runs) and there was no trip match result, bump the result down in the telematics

The key ensemble take away is

-  Do not ensemble any probabilities, instead what you are ensembling is their order in the AUC ranking.   I.e. sort both sets of results based on their probabilities, and then ensemble if it is ranked 1, 2, 50,000 etc

### 3
**首先进行RDP简化，针对某个司机的所有路程，获得本次里程与其他不属于该司机的里程的最大连续匹配。**<br/>
First, process RDP algorithm on trips.Then, for each trip of driver X, get the maximum consecutives matches this trip has with some others trips which don't belong to X
### 4

**利用RDP算法对里程进行简化，定义一个转弯的阈值，以及最小转弯次数，然后将超过阈值的转弯所分割出的距离合并成一个列表，同时再建一张表记录转弯的大小。根据所分割的距离，和转弯角度去进行匹配。（标准化后的均方误？）**<br/>
Simplify trips using RDP (Ramer–Douglas–Peucker), then create just a list of the distances between turns over a threshold along with a second list giving the size of the turn - ignore right or left.  This is a great first pass - very quick and you can weed out a lot of trips this way  I would compare forward and backward (this solved the starting from 0,0 every time bit) just looking at distance between turns, then compare the size of the turn within a threshold (this ignores mirroring) - all together that took care of rotation as well. 

将待匹配的转弯角度变宽从而获得更多匹配项，利用DTW度量里程（DTW是衡量两个时间序列相似度的方法，只有一个里程如何度量？）<br/>

将所有点映射到一维空间，利用DTW去度量。<br/>  


**基于转弯的坐标对里程进行分段匹配并对齐，然后进行SVD，然后将分解的结果对齐子里程，计算所有点的RSME** <br/>
Align the matched sub-trips based on the locations of the turns, and use Singular Value Decomposition to align the sub-trips and get the RSME error for the points.<br/>

根据匹配结果构造近邻树，然后对已匹配的里程进行聚类。根据匹配里程所在聚类簇的规模对进行特征选择。 <br/> 


**用RDP对路线进行简化，对齐并并进行SVD分解，计算子里程的RSME，然后将整个里程对齐，进行而获得整个路程从开始到结束的距离测度。**<br/>
 Use use RDP turn / angle approximations and SVD for alignment, score with RSME for the sub-trip, then align the full trip and measure the distance between the start and end of the trip.<br/>
如果能获得一个里程到另一个里程的近邻树，那就可以rotate and align 所有里程到一个簇里。通过
观察司机匹配图从而调整RSME的参数以及里程的最短距离。如果最小距离大于某个确定的阈值，就
忽略此次匹配。  

***用RDP进行简化，将每次转弯间的路程和转弯的角度记录下来，增加稍大或稍小角度的冗余，前向或后向进行匹配，利用SVD进行对齐,比较对齐的结果,记录每个里程匹配的个数，然后将这些特征进行机器学习。***<br/>
 RDP to simplify the trip, break up into distances between turns (rounded to the nearest 100 meters), record turn sizes (not directions), look for matches forward, backward and with some allowance for slightly larger / smaller turns, then align using SVD, compare the alignment, keep track of how many matches a trip has, and in the end feed that feature fed into a ML algorithm

图聚类簇：任意一个司机都的图簇都包括多个里程，里程间并不完全一致，但里程间存在回路，即可以由一个里程去往另一个里程。如果能选择一个最具连接性的里程，那么就可以将其他里程旋转和对齐到这个里程上。如此，就可以针对每个司机定制路线图。路线图的总长度将超过30km。虽然对于长里程，小的初始误差会累积。  
**一个里程与其他里程连续5次转弯的匹配次数（转弯间的距离与转弯角度），以RSME计算子里程的匹配次数作为额外的特征。post train boost(把匹配结果\*1.1)**<br/>
in the end settled on the number of times a stretch of 5 turns were matched (distance between turns and turn size) with other trips。For the last approach we used the RSME score to count the number of sub-trip matches and used these as additional features.
### 5
**首先进行考虑转弯的修正RDP（确定起始点）简化，加入速度、加速度等特征提升算法。**
### 6
**对每个里程，创建一个包含每x米瞬时转向的向量（以不同的滞后去捕捉50/100米停车的里程）；然后根据累计角度的查分的方差为里程的相似性进行打分；将一组里程与每个司机的每个里程比较，在这个阶段会得的不匹配的里程得分；最后对于每个司机，去计算200个里程得分，并与以获得的里程分布比较，并修正概率。**<br/>
For each trip I created the vector of the instantaneous rotations of every x meters (with different lags - to capture trips where you need 50/100 meters to park).

Then I implemented a similarity Score between two vectors (based on the variance of the difference of the cumulated angle).

I compared a group of trips with exactly one trip per driver. At this stage, I had a Score of trips that (a priori) did not match.

Finally for each driver I computed the Score of the 200 trips and compared it to the distribution I just had... and ajusted the probas in consequence.

### 
把x-y坐标转化为距离-角度坐标，以便旋转。对每个里程对，把最长的里程去掉尾部的一部分。然后修正角度，用相关系数比较曲线的点。将里程对沿距离轴进行小的偏移，去测试偏移是否有强的相关得分。<br/>
###
对里程进行再抽样，创建5米的分割里程；计算里程的交叉相关系数；交叉相关系数能够识别相似里程，以及与大里程相似的小里程。



## 任意两段里程的相似度
$$\sigma_{1,2}=\phi_1\frac{|DT_1-DT_2|}{DT_1+DT_2}+\phi_2\frac{|TN_1-TN_2|}{TN_1+TN_2+1}+\phi_3\frac{|SD_1-SD_2|}{SD_1+SD_2+1}+\phi_4\frac{|ST_1-ST_2|}{ST_1+ST_2+1}$$
其中，DT代表距离；  
TN代表转弯次数；  
SD代表平均停车时间；  
SP代表停车次数；  
**并且DT应赋予较高的权重（需要人工选取），$\sigma_{1,2}$越小代表越相似**。

### 包含驾驶者行为的相似程度
**1、 把急加速、急减速、急转弯作为特征加入$\sigma_{1,2}中$**


## 一段时间内人出行的规律程度
**1、 选取一段时间内多次出行的距离、*行驶方向*、转弯次数、平均停车时间、停车次数等，（进行标准化）**  
**2、 利用k均值算法进行聚类，假设各类所包含样本的个数由多到少依次为$C_1,C_2,C_3,...,C_k(k=5?)$**  
**3、 $\sigma=\frac{C_{k-1}+C_k}{C_1+C_2+...+C_{k-2}}$衡量出行的规律程度，越小越规律**

### 以上下班为主出行相似度
**选取一段时间内多次出行的距离、行驶方向，分别进行标准化，以标准化后的方差之和作为衡量标准，越小越相似。**
